In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [4]:
pc = Pinecone(api_key='')
pc.create_index(
    name='exercise-db', dimension=1536, metric='cosine', spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

In [10]:
import requests

url = "https://exercisedb.p.rapidapi.com/exercises"

querystring = {"limit":"1400","offset":"0"}

headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "exercisedb.p.rapidapi.com"
}

exercise_data = requests.get(url, headers=headers, params=querystring)



In [9]:
%pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
exercise_data = exercise_data.json()

In [32]:
import openai
processed_data = []
openai.api_key = ''
index = pc.Index('exercise-db')
batch_size = 100
for exercise in exercise_data:
    # Combine relevant fields into a single string for embedding
    combined_text = f"{exercise['name']} {exercise['target']} {' '.join(exercise['equipment'])}"
    
    # Get the embedding for the combined text
    response = openai.embeddings.create(
        input=combined_text,
        model='text-embedding-3-small'
    )
    embedding = response.data[0].embedding
    
    # Prepare the data for Pinecone
    processed_data.append({
        'id': exercise['id'],
        'values': embedding,
        'metadata': {
            'name': exercise['name'],
            'target': exercise['target'],
            'secondaryMuscles': exercise['secondaryMuscles'],
            'instructions': exercise['instructions'],
            'equipment': exercise['equipment']
        }
    })
    # Upsert in batches
    if len(processed_data) >= batch_size:
        index.upsert(vectors=processed_data)
        processed_data = []  # Clear the list after upserting

# Upsert any remaining data
if processed_data:
    index.upsert(vectors=processed_data)


# # Index the data in Pinecone
# pc.upsert(vectors=processed_data)

# print("Data indexed successfully!")